In [1]:
import random
import time
import math
import numpy as np

POPULATION_SIZE = 100
GENERATIONS = 500
MUTATION_RATE = 0.1

class Edge:
    def __init__(self, city1, city2, distance):
        self.city1 = city1
        self.city2 = city2
        self.distance = distance

edges = []
distance_matrix = {}
CITY_COUNT = 0

def load_edges(filename):
    global CITY_COUNT
    with open(filename, 'r') as file:
        CITY_COUNT, edge_count = map(int, file.readline().split())
        for _ in range(edge_count):
            city1, city2, distance = map(int, file.readline().split())
            edges.append(Edge(city1, city2, distance))
            if city1 not in distance_matrix:
                distance_matrix[city1] = {}
            if city2 not in distance_matrix:
                distance_matrix[city2] = {}
            distance_matrix[city1][city2] = distance
            distance_matrix[city2][city1] = distance  # Because the graph is undirected

def route_distance(route):
    total_distance = 0
    for i in range(len(route) - 1):
        total_distance += distance_matrix[route[i]][route[i + 1]]
    total_distance += distance_matrix[route[-1]][route[0]]
    return total_distance

def average_distance(distances):
    return sum(distances) / len(distances)

def standard_deviation(distances, mean):
    return math.sqrt(sum((x - mean) ** 2 for x in distances) / len(distances))

def generate_individual(n):
    individual = list(range(1, n + 1))
    random.shuffle(individual)
    return individual

def generate_population(population_size, city_count):
    return [generate_individual(city_count) for _ in range(population_size)]

def crossover(parent1, parent2):
    start = random.randint(0, len(parent1) - 1)
    end = start + random.randint(0, len(parent1) - start)

    child = [-1] * len(parent1)
    child[start:end] = parent1[start:end]

    current = 0
    for city in parent2:
        if city not in child:
            while child[current] != -1:
                current += 1
            child[current] = city

    return child

def mutate(individual):
    if random.random() < MUTATION_RATE:
        i, j = random.sample(range(len(individual)), 2)
        individual[i], individual[j] = individual[j], individual[i]

def tournament_selection(population, fitness):
    tournament_size = 5
    best = random.randint(0, len(population) - 1)
    for _ in range(tournament_size - 1):
        contender = random.randint(0, len(population) - 1)
        if fitness[contender] < fitness[best]:
            best = contender
    return population[best]

if __name__ == '__main__':
    random.seed(time.time())

    # Load data from file
    load_edges("C:\\Users\\mcmys\\source\\repos\\ConsoleApplication6\\ConsoleApplication6\\edges1000.txt")

    population = generate_population(POPULATION_SIZE, CITY_COUNT)
    fitness = [0] * POPULATION_SIZE

    with open("genetic_algorithm_log.txt", 'w') as log_file:
        start_time = time.time()

        for generation in range(GENERATIONS):
            gen_start_time = time.time()

            for i in range(POPULATION_SIZE):
                fitness[i] = route_distance(population[i])

            new_population = []

            mutation_count = 0
            crossover_count = 0

            for _ in range(POPULATION_SIZE // 2):
                parent1 = tournament_selection(population, fitness)
                parent2 = tournament_selection(population, fitness)

                child1 = crossover(parent1, parent2)
                child2 = crossover(parent2, parent1)
                crossover_count += 2

                mutate(child1)
                mutate(child2)
                mutation_count += 2

                new_population.extend([child1, child2])

            fitness = [route_distance(individual) for individual in new_population]

            new_population.sort(key=route_distance)
            population = new_population[:POPULATION_SIZE]

            gen_end_time = time.time()
            gen_time = gen_end_time - gen_start_time

            best_fitness = route_distance(population[0])
            worst_fitness = route_distance(population[-1])
            avg_fitness = average_distance(fitness)
            std_dev_fitness = standard_deviation(fitness, avg_fitness)

            if generation % 10 == 0 or generation == GENERATIONS - 1:
                print(f"Generation {generation}: Best = {best_fitness} (Time: {gen_time:.2f}s)")
                log_file.write(f"Generation {generation}: Best = {best_fitness} (Time: {gen_time:.2f}s)\n")
                log_file.write(f"Worst = {worst_fitness}, Average = {avg_fitness}, StdDev = {std_dev_fitness}\n")
                log_file.write(f"Mutations = {mutation_count}, Crossovers = {crossover_count}\n")

        end_time = time.time()
        total_time = end_time - start_time

        best_index = 0
        best_route = population[best_index]
        best_distance = route_distance(best_route)

        print("Best route:", ' '.join(map(str, best_route)))
        print(f"Best distance: {best_distance}")
        print(f"Total execution time: {total_time:.2f}s")

        log_file.write("Best route: " + ' '.join(map(str, best_route)) + "\n")
        log_file.write(f"Best distance: {best_distance}\n")
        log_file.write(f"Total execution time: {total_time:.2f}s\n")


Generation 0: Best = 474942 (Time: 0.87s)
Generation 10: Best = 446613 (Time: 0.88s)
Generation 20: Best = 419891 (Time: 0.94s)
Generation 30: Best = 405478 (Time: 0.88s)
Generation 40: Best = 391817 (Time: 0.85s)
Generation 50: Best = 381141 (Time: 0.87s)
Generation 60: Best = 367154 (Time: 0.85s)
Generation 70: Best = 360188 (Time: 0.87s)
Generation 80: Best = 352352 (Time: 0.84s)
Generation 90: Best = 344131 (Time: 0.86s)
Generation 100: Best = 337212 (Time: 0.85s)
Generation 110: Best = 331485 (Time: 1.01s)
Generation 120: Best = 326226 (Time: 0.89s)
Generation 130: Best = 318583 (Time: 2.37s)
Generation 140: Best = 312546 (Time: 0.87s)
Generation 150: Best = 306664 (Time: 0.95s)
Generation 160: Best = 302363 (Time: 0.82s)
Generation 170: Best = 298876 (Time: 0.83s)
Generation 180: Best = 294653 (Time: 1.07s)
Generation 190: Best = 290734 (Time: 0.95s)
Generation 200: Best = 284736 (Time: 1.13s)
Generation 210: Best = 278830 (Time: 1.53s)
Generation 220: Best = 273638 (Time: 1.03s)